Zde hledáme Wikidata IDs toho autorstva, které je nemá vyplněné v databázi autorit.

In [2]:
import gc
import os
import re
import json
import requests
import pandas as pd

In [3]:
kam = 'data_raw/wikidata_doplneni'
if not os.path.exists(kam):
    os.makedirs(kam)

In [4]:
vsechny_jsony = [f for f in os.listdir(kam)]

In [5]:
vyhledane = set([x.split("_")[0] for x in vsechny_jsony])

In [6]:
df = pd.read_parquet(os.path.join("data/cnb_sloupce","100.parquet"))

In [7]:
df = df.explode('100_7')

In [8]:
df['100_7'].nunique()

172624

In [9]:
df.groupby('100_7').size().nlargest(1000)

100_7
jx20060515016    2987
jx20050830020    2373
jn20001103529    1672
jn19981000826    1487
jk01120043       1394
                 ... 
jk01073231         65
jk01080598         65
jk01092978         65
jk01121431         65
jk01131204         65
Length: 1000, dtype: int64

In [10]:
opakovane = df.groupby('100_7').size()
opakovane = opakovane.sort_values(ascending=False)
opakovane = opakovane[opakovane > 1].index.to_list()
len(opakovane)

91311

In [11]:
autority = pd.read_parquet(os.path.join("data","aut_vyber.parquet"))

In [12]:
autority

,024_2,024_a,046_f,046_g,100_7,100_a,100_d,100_ind1,110_a,370_a,...,411_a,430_a,450_a,500_a,500_i,500_ind1,550_7,678_a,856_u,leader
001,,,,,,,,,,,,,,,,,,,,,
jk01010001,None,None,[1931],None,[jk01010001],"[Abraham, Jiří,]",[1931-],[1],None,None,...,None,None,None,None,None,None,None,"[Narozen 14.12.1931 v Osvračíně. MUDr., odborn...",None,nz a22 n 4500
jk01010002,None,None,[1890],[1933],[jk01010002],"[Abraham, Josef,]",[1890-1933],[1],None,None,...,None,None,None,None,None,None,None,[Narozen 19.11.1890 v Osvračíně u Horšova Týna...,None,nz a22 n 4500
jk01010003,None,None,[1957],None,[jk01010003],"[Abraham, Pavel,]",[1957-],[1],None,None,...,None,None,None,None,None,None,None,[Narozen roku 1957. Student filosofické fakult...,None,nz a22 n 4500
jk01010004,None,None,[1889],[1963],[jk01010004],"[Absolon, Josef,]",[1889-1963],[1],None,"[Odrlice, Senice na Hané, Česko]",...,None,None,None,None,None,None,None,"[Narozen 27. 1. 1889 v Odrlicích na Moravě, ze...",None,cz a22 n 4500
jk01010005,"[isni, wikidata]","[0000000109542912, Q1063371]",[1877],[1960],[jk01010005],"[Absolon, Karel,]",[1877-1960],[1],None,None,...,None,None,None,None,None,None,None,"[Narozen 16.6.1877 v Boskovicích, zemřel 6.10....",None,cz a22 n 4500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
jo20251251608,[isni],[0000000061570745],[1951],None,[jo20251251608],"[Biemans, J. A. A. M.,]",[1951-],[1],None,None,...,None,None,None,None,None,None,None,[Kurátor sbírky rukopisů univerzitní knihovny ...,None,nz a22 n 4500
xx0328654,None,None,[1986],None,[xx0328654],"[Lobe, Max,]",[1986-],[1],None,"[Douala, Kamerun]",...,None,None,None,None,None,None,None,[Švýcarský novinář a spisovatel kamerunského p...,None,nz a22 n 4500
aun20251251610,None,None,None,None,[aun20251251610],"[Verne, Jules,]",[1828-1905.],[1],None,None,...,None,None,None,None,None,None,None,None,None,nz a22 n 4500


In [13]:
autority = autority[autority['024_2'].notnull()]
autority = autority[autority['024_a'].notnull()]

In [14]:
autority = autority.explode(['024_a','024_2'])

In [15]:
autority_s_wikidaty = set(autority[autority['024_2'] == 'wikidata']['024_a'].index.to_list())

In [16]:
chybi = [x for x in opakovane if x not in autority_s_wikidaty]

In [17]:
del df
del autority
gc.collect()

0

In [18]:
len(chybi)

82568

In [19]:
# Wikidata SPARQL endpoint
sparql_endpoint = "https://query.wikidata.org/sparql"

# Function to get Wikidata ID from NL CR AUT ID
def get_wikidata_id(nl_cr_aut_id):
    query = f"""
    SELECT ?item WHERE {{
      ?item wdt:P691 "{nl_cr_aut_id}".
    }}
    """
    headers = {
        "Accept": "application/sparql-results+json",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    }
    response = requests.get(sparql_endpoint, params={"query": query}, headers=headers)
    data = response.json()

    if "results" in data and "bindings" in data["results"]:
        bindings = data["results"]["bindings"]
        if bindings:
            return bindings[0]["item"]["value"].split("/")[-1]
    return None

In [20]:
vyhledane = set(vyhledane)

In [21]:
co_stahovat = [x for x in chybi if x not in vyhledane]

In [22]:
len(co_stahovat)

328

In [23]:
pocitadlo = 0
for nl_cr_aut_id in co_stahovat:
    pocitadlo += 1
    print(f"{pocitadlo:005}: {nl_cr_aut_id}")
    slovnik = {}
    slovnik['100_7'] = nl_cr_aut_id
    wikidata_id = get_wikidata_id(nl_cr_aut_id)
    if wikidata_id:
        slovnik['024_a'] = wikidata_id
    else:
        slovnik['024_a'] = ""
    with open(os.path.join(kam, f"{nl_cr_aut_id}_{slovnik['024_a']}.json"), "w+", encoding="utf-8") as file:
        file.write(json.dumps(slovnik))

00001: jo20241247980
00002: jo20251249220
00003: xx0328621
00004: jo20251248910
00005: mzk20251251427
00006: jo20251248190
00007: jo20221171824
00008: skuk0005980
00009: xx0079318
00010: jo20251248909
00011: jo2003181510
00012: mzk20241247404
00013: mzk20211130044
00014: xx0280130
00015: xx0279229
00016: xx0249491
00017: xx0307172
00018: jo20221143435
00019: xx0124269
00020: mzk2011640821
00021: jk01072252
00022: xx0302480
00023: xx0308902
00024: xx0281697
00025: mzk2010582835
00026: xx0323214
00027: mzk2009496352
00028: xx0310712
00029: hka2011672886
00030: hka2014817308
00031: ola20211133285
00032: xx0091638
00033: js20100106003
00034: jx20110216020
00035: xx0273035
00036: mzk20221145066
00037: mzk20221171290
00038: xx0225255
00039: mzk2005289983
00040: jn19990209235
00041: mzk2004265209
00042: mzk2017941811
00043: mzk2008448599
00044: mzk2004261086
00045: mzk20181006095
00046: xx0031969
00047: xx0010286
00048: mzk20181015238
00049: mzk20181010787
00050: jx20080723014
00051: xx006654